# SQL ПЛЭЙБУК: Полный справочник для собеседований и работы

**PostgreSQL** — стандарт для аналитики + явные различия с MySQL.

## СОДЕРЖАНИЕ
[1. Фундаментальные основы](#РАЗДЕЛ-1:-Фундаментальные-основы)  
[2. Выборка и фильтрация данных](#раздел-2-выборка-и-фильтрация-данных)  
[3. Агрегация и группировка](#раздел-3-агрегация-и-группировка)  
[4. Соединения таблиц (JOINs)](#раздел-4-соединения-таблиц-joins)  
[5. Подзапросы (Subqueries)](#раздел-5-подзапросы-subqueries)  
[6. CTE (Common Table Expressions)](#раздел-6-cte-common-table-expressions)  
[7. Оконные функции](#раздел-7-оконные-функции)  
[8. Оптимизация и индексы](#раздел-8-оптимизация-и-индексы)  
[9. Транзакции и управление](#раздел-9-транзакции-и-управление)  
[10. Работа с датами и строками](#раздел-10-работа-с-датами-и-строками)  
[11. Продвинутые темы](#раздел-11-продвинутые-темы)  
[12. Практические паттерны и задачи](#раздел-12-практические-паттерны-и-задачи)  
[Приложения](#приложения)

---
**Источники:** PostgreSQL docs, MySQL docs, LeetCode SQL.  
*Добавляйте свои шаблоны через PR.*

## РАЗДЕЛ 1: Фундаментальные основы

### 1.1. SQL и реляционные базы данных

**Что такое SQL?**
SQL (Structured Query Language) — декларативный язык программирования для работы с реляционными базами данных. Позволяет:
- Создавать и изменять структуру БД (DDL — Data Definition Language)
- Управлять данными (DML — Data Manipulation Language) 
- Контролировать доступ (DCL — Data Control Language)
- Управлять транзакциями (TCL — Transaction Control Language)

**Отличие SQL от MySQL?**
- **SQL** — стандартизированный язык запросов (ANSI/ISO стандарт)
- **MySQL** — конкретная СУБД (система управления базами данных), которая реализует стандарт SQL

**Реляционная база данных** — данные организованы в таблицы (отношения), связанные ключами. Основные понятия:
- Таблица (Table) — совокупность строк и столбцов
- Строка (Row/Record) — одна запись в таблице
- Столбец (Column/Field) — атрибут записи
- Первичный ключ (Primary Key) — уникальный идентификатор строки
- Внешний ключ (Foreign Key) — ссылка на первичный ключ другой таблицы

### 1.2. Типы данных

**Основные категории типов данных:**

| Категория | Примеры | Описание |
|-----------|---------|----------|
| Строковые | `CHAR`, `VARCHAR`, `TEXT` | Текстовые данные |
| Числовые | `INT`, `DECIMAL`, `FLOAT` | Числа |
| Дата/время | `DATE`, `TIME`, `TIMESTAMP` | Даты и время |
| Логические | `BOOLEAN` | Логические значения |
| Бинарные | `BLOB`, `BINARY` | Бинарные данные |
| JSON | `JSON`, `JSONB` | JSON документы |

**Разница между CHAR и VARCHAR**
```sql
-- CHAR - фиксированная длина (дополняется пробелами)
CREATE TABLE example_char (code CHAR(10));  -- 'ABC' -> 'ABC       '

-- VARCHAR - переменная длина (хранится только фактический размер)  
CREATE TABLE example_varchar (name VARCHAR(100));  -- 'Иван' -> 'Иван'

-- Рекомендации:
-- Использовать VARCHAR для текстовых полей
-- CHAR только для кодов фиксированной длины (ISO коды, аббревиатуры)
```
**Выбор типа данных:**
```sql
- INT для целых чисел (id, количество)
- DECIMAL(p,s) для денежных значений (p — точность, s — масштаб)
- VARCHAR(n) для текста, где n — максимальная длина
- DATE для дат без времени
- TIMESTAMP для даты и времени
- JSON / JSONB для неструктурированных данных (PostgreSQL: JSONB для бинарного хранения)
```

### **1.3. Ограничения (Constraints)**

#### **Что такое `PRIMARY KEY`?**
Первичный ключ — столбец (или комбинация столбцов), уникально идентифицирующий каждую строку:
* Гарантирует **уникальность** значений
* Не допускает **NULL** значений
* Автоматически создает индекс (обычно кластеризованный)
* В таблице может быть только **один** `PRIMARY KEY`

```sql
-- Создание первичного ключа
CREATE TABLE users (
    id INT PRIMARY KEY,
    email VARCHAR(100) NOT NULL UNIQUE
);
CREATE TABLE orders (
    order_id INT,
    user_id INT,
    PRIMARY KEY (order_id, user_id)
);
```

#### **Что такое `FOREIGN KEY`?**
Внешний ключ — ограничение, обеспечивающее ссылочную целостность между таблицами:
* Связывает поле в дочерней таблице с `PRIMARY KEY` в родительской
* Предотвращает удаление связанных данных (`CASCADE`, `SET NULL`, `RESTRICT`)
* Обеспечивает целостность отношений

```sql
-- Создание внешнего ключа
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    user_id INT,
    FOREIGN KEY (user_id) REFERENCES users(id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);
-- Проверка внешних ключей (MySQL)
SHOW CREATE TABLE orders;
```

#### **Типы ограничений:**
| Ограничение | Описание | Синтаксис |
| :--- | :--- | :--- |
| `NOT NULL` | Запрещает NULL | `name VARCHAR(50) NOT NULL` |
| `UNIQUE` | Гарантирует уникальность | `email VARCHAR(100) UNIQUE` |
| `PRIMARY KEY` | Первичный ключ | `id INT PRIMARY KEY` |
| `FOREIGN KEY` | Внешний ключ | `FOREIGN KEY (col) REFERENCES table(col)` |
| `CHECK` | Проверка условия | `age INT CHECK (age >= 18)` |
| `DEFAULT` | Значение по умолчанию | `created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP` |

#### **Практический пример:**
```sql
CREATE TABLE employees (
    id INT PRIMARY KEY AUTO_INCREMENT,
    employee_code CHAR(10) UNIQUE NOT NULL,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    age INT CHECK (age >= 18 AND age <= 70),
    department_id INT,
    salary DECIMAL(10,2) DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (department_id) REFERENCES departments(id)
        ON DELETE SET NULL
);
```

### **1.4. CRUD операции**

#### **`CREATE` — создание таблиц**

```sql
-- Базовая структура
CREATE TABLE table_name (
    column1 datatype constraints,
    column2 datatype constraints,
    ...
);

-- Пример
CREATE TABLE products (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price DECIMAL(10,2) CHECK (price >= 0),
    category VARCHAR(50),
    in_stock BOOLEAN DEFAULT TRUE,
    created_at DATETIME DEFAULT NOW()
);

-- Копирование структуры
CREATE TABLE products_backup LIKE products;  -- MySQL
CREATE TABLE products_backup AS SELECT * FROM products WHERE 1=0;  -- кросс-платформенный
```

#### **`ALTER` — изменение таблиц**

```sql
-- Добавить столбец
ALTER TABLE employees ADD COLUMN birth_date DATE;

-- Изменить тип столбца
ALTER TABLE employees MODIFY COLUMN name VARCHAR(150);  -- MySQL
ALTER TABLE employees ALTER COLUMN name TYPE VARCHAR(150);  -- PostgreSQL

-- Удалить столбец  
ALTER TABLE employees DROP COLUMN middle_name;

-- Добавить ограничение
ALTER TABLE employees ADD CONSTRAINT chk_age CHECK (age >= 18);

-- Переименовать таблицу
ALTER TABLE old_name RENAME TO new_name;
```

#### **`INSERT` — добавление данных**

```sql
-- Вставка одной строки
INSERT INTO users (name, email, age) 
VALUES ('Иван Иванов', 'ivan@example.com', 30);

-- Вставка нескольких строк
INSERT INTO products (name, price, category) VALUES
    ('Ноутбук', 1500.00, 'Электроника'),
    ('Стул', 200.00, 'Мебель'),
    ('Книга', 30.00, 'Книги');

-- Вставка из другой таблицы
INSERT INTO users_archive (id, name, email)
SELECT id, name, email 
FROM users 
WHERE created_at < '2023-01-01';
```

#### **`SELECT` — выборка данных**

```sql
-- Получить все данные
SELECT * FROM employees;

-- Выбрать конкретные столбцы
SELECT name, email, salary FROM employees;

-- Выбрать уникальные значения
SELECT DISTINCT department FROM employees;
```

#### **`UPDATE` — обновление данных**

```sql
-- Обновить все строки
UPDATE products SET price = price * 1.1;  -- увеличить все цены на 10%

-- Обновить с условием
UPDATE employees 
SET salary = salary * 1.15, 
    updated_at = NOW()
WHERE department = 'IT' 
  AND performance_rating > 4.0;

-- Обновить из другой таблицы (MySQL)
UPDATE employees e
JOIN salary_changes s ON e.id = s.employee_id
SET e.salary = s.new_salary
WHERE s.effective_date = '2025-01-01';
```

#### **`DELETE` — удаление данных**

```sql
-- Удалить с условием
DELETE FROM orders 
WHERE status = 'cancelled' 
  AND created_at < '2024-01-01';

-- Удалить все данные (осторожно!)
DELETE FROM temp_logs;  -- можно откатить, медленно
```

### **1.5. Разница между DELETE, TRUNCATE и DROP**

#### **Сравнение операций:**

| Операция | Скорость | Возврат (ROLLBACK) | Автоинкремент | Где использовать |
| :--- | :--- | :--- | :--- | :--- |
| `DELETE` | Медленно | **Да** | Сохраняется | Удаление строк с условием `WHERE` |
| `TRUNCATE` | Быстро | **Нет*** | Сбрасывается | Удаление **ВСЕХ** строк, сброс таблицы |
| `DROP` | Мгновенно | Нет | Удаляется | Удаление всей таблицы **со структурой** |

\* В PostgreSQL `TRUNCATE` можно откатить, если он выполнен внутри транзакции.

#### **Примеры команд:**

```sql
-- DELETE: удаление с условием, можно откатить
BEGIN TRANSACTION;
DELETE FROM users WHERE age < 18;
-- ROLLBACK; -- откатить удаление
COMMIT;

-- TRUNCATE: удаление всех данных, быстро (нельзя использовать WHERE)
TRUNCATE TABLE session_logs;

-- DROP: удаление таблицы полностью (удаляет и данные, и структуру)
DROP TABLE temporary_data;

-- Каскадное удаление
DROP TABLE users CASCADE;  -- удаляет таблицу 'users' и все зависимые от неё объекты
```

#### **Важные нюансы:**

*   **`TRUNCATE`** — это операция **DDL** (Data Definition Language), а не DML (как `DELETE`).
*   **`DELETE`** активирует триггеры удаления, **`TRUNCATE`** — **нет**.
*   **`TRUNCATE`** обычно требует больших привилегий (часто доступно только у администратора БД - DBA).
*   В **PostgreSQL** `TRUNCATE` можно откатить (`ROLLBACK`), если он выполнен внутри транзакции.

### **Антипаттерны раздела 1**

#### ❌ **Не использовать первичные ключи**

```sql
-- ПЛОХО: таблица без первичного ключа
CREATE TABLE logs (
    message TEXT,
    created_at TIMESTAMP
);
-- Проблемы: дубликаты, сложность обновления, медленные JOIN

-- ✅ ХОРОШО: всегда добавлять первичный ключ
CREATE TABLE logs (
    id INT PRIMARY KEY AUTO_INCREMENT,
    message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

#### ❌ **Использовать VARCHAR без указания длины**

```sql
-- ПЛОХО: неограниченная длина (MySQL создаст TEXT)
CREATE TABLE users (
    name VARCHAR,  -- ошибка в MySQL, требуется указать длину
    email VARCHAR
);

-- ✅ ХОРОШО: указывать разумную длину
CREATE TABLE users (
    name VARCHAR(100),
    email VARCHAR(150)
);
```

#### ❌ **Хранить несколько значений в одном поле**

```sql
-- ПЛОХО: нарушение первой нормальной формы
CREATE TABLE orders (
    id INT PRIMARY KEY,
    product_ids VARCHAR(255)  -- '1,5,23,42'
);

-- ✅ ХОРОШО: использовать связующую таблицу
CREATE TABLE orders (
    id INT PRIMARY KEY
);

CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
```

#### ❌ **Использовать SELECT * в продакшене**

```sql
-- ПЛОХО: выбирать все столбцы
SELECT * FROM users WHERE id = 1;

-- ✅ ХОРОШО: выбирать только нужные столбцы
SELECT id, name, email FROM users WHERE id = 1;
-- Преимущества: меньше данных по сети, лучше кэширование, индексы работают эффективнее
```

### **Лучшие практики раздела 1**

#### ✅ **Всегда использовать первичные ключи**
*   Даже для логов и временных данных
*   Автоинкремент для таблиц с частыми вставками
*   UUID для распределенных систем

#### ✅ **Выбирать подходящие типы данных**
*   `INT` для счетчиков и идентификаторов
*   `DECIMAL` для денежных значений (не `FLOAT`)
*   `VARCHAR` с разумной длиной для текста
*   `DATETIME`/`TIMESTAMP` для меток времени

#### ✅ **Использовать ограничения на уровне БД**
*   `NOT NULL` для обязательных полей
*   `UNIQUE` для email, телефонных номеров
*   `FOREIGN KEY` для поддержания целостности
*   `CHECK` для валидации бизнес-правил

#### ✅ **Следовать соглашениям об именовании**

```sql
-- Пример хороших имен:
users                -- таблицы во множественном числе
id                   -- первичный ключ
user_id              -- внешний ключ
created_at, updated_at  -- временные метки
is_active, has_access  -- булевы флаги
```

### **Различия MySQL и PostgreSQL**

| Концепция | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Тип строки по умолчанию** | `VARCHAR(255)` | Без ограничения длины |
| **Логический тип** | `BOOLEAN` (синоним `TINYINT(1)`) | Нативный `BOOLEAN` |
| **Автоинкремент** | `AUTO_INCREMENT` | `SERIAL` или `GENERATED AS IDENTITY` |
| **Каскадное удаление** | `ON DELETE CASCADE` | `ON DELETE CASCADE` |
| **Проверка длины строки** | `CHECK(LENGTH(name) > 0)` | То же самое |

### **Шпаргалка раздела 1**

#### **Создание таблицы:**

```sql
CREATE TABLE table_name (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(150) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

#### **Основные типы данных:**
*   **`INT`, `BIGINT`** — целые числа
*   **`DECIMAL(10,2)`** — числа с фиксированной точностью
*   **`VARCHAR(n)`** — текст переменной длины
*   **`TEXT`** — длинный текст
*   **`DATE`, `DATETIME`, `TIMESTAMP`** — дата и время
*   **`BOOLEAN`** — истина/ложь

#### **Ограничения:**
*   **`PRIMARY KEY`** — уникальный идентификатор
*   **`FOREIGN KEY`** — связь между таблицами
*   **`NOT NULL`** — обязательное поле
*   **`UNIQUE`** — уникальные значения
*   **`CHECK`** — проверка условия
*   **`DEFAULT`** — значение по умолчанию

#### **Операции с данными:**

```sql
INSERT INTO table (col1, col2) VALUES (val1, val2);
SELECT * FROM table WHERE condition;
UPDATE table SET col = value WHERE condition;
DELETE FROM table WHERE condition;
```

## РАЗДЕЛ 2: Выборка и фильтрация данных

### **2.1. WHERE, ORDER BY, LIMIT/OFFSET**

#### **Базовая структура запроса SELECT**

```sql
SELECT column1, column2, ...        -- Что выбрать
FROM table_name                     -- Откуда выбрать
WHERE condition                     -- Условие фильтрации строк
ORDER BY column1 [ASC|DESC]         -- Сортировка результатов
LIMIT number_of_rows;               -- Ограничение количества строк
```

#### **Ключевое слово WHERE**
Фильтрует строки до их обработки и агрегации.

```sql
-- Простые условия
SELECT * FROM employees WHERE salary > 50000;
SELECT * FROM products WHERE price BETWEEN 20 AND 100;
SELECT * FROM users WHERE country IN ('Россия', 'Беларусь', 'Казахстан');

-- Комбинирование условий
SELECT * FROM orders 
WHERE status = 'completed' 
  AND total_amount > 1000 
  AND order_date >= '2024-01-01';
```

#### **Сортировка с ORDER BY**

```sql
-- Сортировка по одному столбцу (по возрастанию ASC по умолчанию)
SELECT name, salary FROM employees ORDER BY salary;

-- Сортировка по убыванию
SELECT name, salary FROM employees ORDER BY salary DESC;

-- Сортировка по нескольким столбцам
SELECT first_name, last_name, department, salary 
FROM employees 
ORDER BY department ASC, salary DESC;

-- Сортировка по выражению или номеру столбца (не рекомендуется для продакшена)
SELECT name, salary, salary * 0.87 AS salary_after_tax
FROM employees 
ORDER BY 3 DESC;  -- Сортировка по 3-му столбцу (salary_after_tax)
```

#### **Ограничение результатов LIMIT и OFFSET**

```sql
-- Получить первые 10 записей
SELECT * FROM products ORDER BY price DESC LIMIT 10;

-- Пагинация: пропустить 20 записей, взять следующие 10
SELECT * FROM products 
ORDER BY created_at DESC 
LIMIT 10 OFFSET 20;  -- Страница 3 при размере страницы 10

-- Альтернативный синтаксис (MySQL)
SELECT * FROM products LIMIT 20, 10;  -- OFFSET 20, LIMIT 10
```

#### **Порядок выполнения запроса для понимания:**
1.  **`FROM`** — определение таблицы
2.  **`WHERE`** — фильтрация строк
3.  **`SELECT`** — выбор столбцов
4.  **`ORDER BY`** — сортировка
5.  **`LIMIT`/`OFFSET`** — ограничение вывода

### **2.2. Операторы сравнения и логические операторы**

#### **Операторы сравнения:**

| Оператор | Описание | Пример |
| :--- | :--- | :--- |
| `=` | Равно | `WHERE status = 'active'` |
| `<>` или `!=` | Не равно | `WHERE status <> 'deleted'` |
| `<` | Меньше | `WHERE age < 18` |
| `>` | Больше | `WHERE salary > 50000` |
| `<=` | Меньше или равно | `WHERE quantity <= 10` |
| `>=` | Больше или равно | `WHERE score >= 60` |
| `BETWEEN` | В диапазоне (включительно) | `WHERE price BETWEEN 10 AND 100` |
| `IN` | В списке значений | `WHERE country IN ('RU', 'BY', 'KZ')` |
| `LIKE` | Соответствие шаблону | `WHERE name LIKE 'Ив%'` |
| `IS NULL` | Проверка на NULL | `WHERE email IS NULL` |

#### **Логические операторы:**

```sql
-- AND: оба условия истинны
SELECT * FROM users WHERE age >= 18 AND age <= 65;

-- OR: хотя бы одно условие истинно
SELECT * FROM products WHERE category = 'Электроника' OR price > 1000;

-- NOT: отрицание условия
SELECT * FROM orders WHERE NOT status = 'cancelled';

-- Комбинации с использованием скобок для контроля приоритета
SELECT * FROM employees 
WHERE (department = 'IT' AND salary > 80000)
   OR (department = 'Sales' AND commission > 0.1);
```

#### **Приоритет операторов (от высшего к низшему):**
1.  **Скобки `()`**
2.  **Операторы сравнения** (`=`, `<>`, `<`, `>`, и т.д.)
3.  **`NOT`**
4.  **`AND`**
5.  **`OR`**

### **2.3. LIKE и регулярные выражения**

#### **Оператор LIKE для поиска по шаблону:**

```sql
-- %: любое количество любых символов (0 или более)
SELECT * FROM users WHERE name LIKE 'Ив%';     -- Начинается на "Ив" (Иван, Игорь)
SELECT * FROM users WHERE email LIKE '%gmail.com';  -- Заканчивается на "gmail.com"
SELECT * FROM products WHERE name LIKE '%ноутбук%'; -- Содержит "ноутбук"

-- _: один любой символ
SELECT * FROM users WHERE phone LIKE '+7___%';  -- Российские номера (после +7 три любые цифры)
SELECT * FROM products WHERE sku LIKE 'ABC_2024'; -- ABC12024, ABC22024, и т.д.

-- Экранирование специальных символов
SELECT * FROM logs WHERE message LIKE '100\% complete';  -- Поиск "100% complete"
```

#### **Регулярные выражения (REGEXP):**

```sql
-- MySQL: REGEXP
SELECT * FROM users WHERE email REGEXP '^[a-z]+@gmail\.com$';  -- только gmail
SELECT * FROM products WHERE name REGEXP '^(Pro|Premium)';  -- начинается с Pro или Premium

-- PostgreSQL: ~ (регистрозависимый) и ~* (регистронезависимый)
SELECT * FROM users WHERE email ~ '^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$';
SELECT * FROM products WHERE name ~* 'iphone.*(14|15)';  -- iPhone 14 или 15, регистронезависимо

-- Проверка на соответствие шаблону
SELECT 'hello' REGEXP '^h.*o$';  -- MySQL: возвращает 1 (true)
SELECT 'hello' ~ '^h.*o$';       -- PostgreSQL: возвращает true
```

### **2.4. Работа с NULL значениями**

#### **Что такое NULL?**
*   **NULL** означает "отсутствие значения" или "неизвестное значение".
*   **NULL ≠ 0**, NULL ≠ пустая строка `''`, NULL ≠ false.
*   Любая операция с NULL возвращает NULL (`NULL + 5 = NULL`, `NULL = NULL` → `NULL`).

#### **Проверка на NULL:**

```sql
-- ❌ НЕПРАВИЛЬНО (всегда возвращает NULL, что интерпретируется как false)
SELECT * FROM employees WHERE phone = NULL;  -- Не найдет ни одной строки
SELECT * FROM employees WHERE phone != NULL; -- Не найдет ни одной строки

-- ✅ ПРАВИЛЬНО: использовать IS NULL или IS NOT NULL
SELECT * FROM employees WHERE phone IS NULL;
SELECT * FROM employees WHERE phone IS NOT NULL;
```

#### **Функции для работы с NULL:**

```sql
-- COALESCE: возвращает первое не-NULL значение
SELECT 
    name, 
    COALESCE(phone, 'Не указан') AS phone_display,
    COALESCE(email, phone, 'Нет контактов') AS contact
FROM users;

-- IFNULL (MySQL) / ISNULL (SQL Server): аналог COALESCE для двух аргументов
SELECT name, IFNULL(phone, 'N/A') FROM users;  -- MySQL

-- NULLIF: возвращает NULL если значения равны, иначе первое значение
SELECT NULLIF(sale_price, 0) AS effective_price FROM products;  -- Заменяет 0 на NULL

-- Пример практического использования
SELECT 
    COUNT(*) AS total_users,
    COUNT(phone) AS users_with_phone,  -- COUNT игнорирует NULL
    COUNT(DISTINCT email) AS unique_emails
FROM users;
```

#### **NULL в агрегатных функциях:**

```sql
-- Агрегатные функции игнорируют NULL (кроме COUNT(*))
SELECT 
    AVG(commission) AS avg_commission,           -- NULL не учитываются
    SUM(bonus) AS total_bonus,                   -- NULL игнорируются
    MIN(salary) AS min_salary,                   -- NULL игнорируются
    MAX(salary) AS max_salary,                   -- NULL игнорируются
    COUNT(commission) AS employees_with_commission,  -- Только не-NULL
    COUNT(*) AS total_employees                  -- Все строки, включая NULL
FROM employees;
```

#### **Сортировка NULL значений:**

```sql
-- По умолчанию NULL считается наименьшим значением (идут первыми при ASC)
SELECT * FROM products ORDER BY price ASC;  -- NULL будут первыми

-- Явное указание расположения NULL
SELECT * FROM employees 
ORDER BY commission DESC NULLS LAST;  -- NULL в конце

SELECT * FROM employees 
ORDER BY commission ASC NULLS FIRST;  -- NULL в начале (по умолчанию)
```

### **Антипаттерны раздела 2**

#### ❌ **Использование функций в WHERE на индексированных полях**

```sql
-- ПЛОХО: индекс не используется
SELECT * FROM users WHERE YEAR(created_at) = 2024;
SELECT * FROM users WHERE UPPER(name) = 'ИВАН';

-- ✅ ХОРОШО: переписать без функций
SELECT * FROM users WHERE created_at >= '2024-01-01' AND created_at < '2025-01-01';
SELECT * FROM users WHERE name = 'Иван';  -- Если нужен регистронезависимый поиск, используйте COLLATE
```

#### ❌ **Неправильная проверка диапазонов дат**

```sql
-- ПЛОХО: преобразование типов, индекс не используется
SELECT * FROM orders WHERE DATE(order_datetime) = '2024-01-15';

-- ✅ ХОРОШО: использовать диапазон
SELECT * FROM orders 
WHERE order_datetime >= '2024-01-15 00:00:00' 
  AND order_datetime < '2024-01-16 00:00:00';
```

#### ❌ **Использование OR вместо IN для многих значений**

```sql
-- ПЛОХО: нечитаемо, может быть медленнее
SELECT * FROM products 
WHERE category = 'Электроника' 
   OR category = 'Бытовая техника'
   OR category = 'Мебель'
   OR category = 'Книги';

-- ✅ ХОРОШО: использовать IN
SELECT * FROM products 
WHERE category IN ('Электроника', 'Бытовая техника', 'Мебель', 'Книги');

-- Еще лучше: использовать временную таблицу или таблицу-справочник
```

#### ❌ **SELECT * в больших таблицах**

```sql
-- ПЛОХО: выборка всех столбцов, включая тяжелые (TEXT, BLOB)
SELECT * FROM products WHERE category = 'Электроника';

-- ✅ ХОРОШО: выбирать только необходимые столбцы
SELECT id, name, price, category 
FROM products 
WHERE category = 'Электроника';
```

### **Лучшие практики раздела 2**

#### ✅ **Всегда использовать ORDER BY с LIMIT**

```sql
-- Без ORDER BY результат непредсказуем
SELECT * FROM products LIMIT 10;  -- ❌ Какие 10? Случайные!

-- С ORDER BY результат детерминирован
SELECT * FROM products ORDER BY created_at DESC LIMIT 10;  -- ✅ Последние 10
```

#### ✅ **Правильное экранирование пользовательского ввода**

```sql
-- Защита от SQL-инъекций
-- ❌ ОПАСНО (SQL-инъекция возможна):
"SELECT * FROM users WHERE name = '" + user_input + "'"

-- ✅ БЕЗОПАСНО: использовать параметризованные запросы
-- В приложении:
cursor.execute("SELECT * FROM users WHERE name = %s", (user_input,))

-- Или использовать ORM, который экранирует ввод автоматически
```

#### ✅ **Использование EXPLAIN для анализа запросов**

```sql
-- Перед оптимизацией всегда смотрите план выполнения
EXPLAIN SELECT * FROM users WHERE email LIKE '%@gmail.com';

-- В MySQL можно добавить FORMAT=JSON для детальной информации
EXPLAIN FORMAT=JSON SELECT * FROM large_table WHERE date_column > '2024-01-01';
```

#### ✅ **Создание индексов для часто фильтруемых полей**

```sql
-- Для ускорения WHERE и ORDER BY
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
CREATE INDEX idx_products_category_price ON products(category, price DESC);

-- Для ускорения поиска по префиксу
CREATE INDEX idx_users_name_prefix ON users(name(10));  -- Первые 10 символов (MySQL)
```

### **Различия MySQL и PostgreSQL**

| Операция | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Конкатенация строк** | `CONCAT('A', 'B')` или `'A' ‖ 'B'` с `SET sql_mode='PIPES_AS_CONCAT'` | `'A' ‖ 'B'` или `CONCAT('A', 'B')` |
| **Регулярные выражения** | `REGEXP`, `RLIKE` | `~`, `~*`, `!~`, `!~*` |
| **LIMIT/OFFSET** | `LIMIT 10 OFFSET 20` или `LIMIT 20, 10` | Только `LIMIT 10 OFFSET 20` |`
| **Проверка на NULL в WHERE** | `WHERE column IS NULL` | `WHERE column IS NULL` |
| **Сортировка NULL** | `ORDER BY column ASC` (NULL первые) | `ORDER BY column ASC NULLS FIRST\|LAST` |
| **LIKE с escape** | `LIKE '100\%' ESCAPE '\'` | `LIKE '100\%' ESCAPE '\'` |
| **ILIKE (регистронезависимый)** | Не поддерживается, использовать `LIKE` или `REGEXP` | Поддерживается `ILIKE` |

### **Шпаргалка раздела 2**

#### **Базовый запрос с фильтрацией:**

```sql
SELECT column1, column2
FROM table
WHERE condition
ORDER BY column1 DESC
LIMIT 10 OFFSET 20;
```

#### **Основные операторы WHERE:**
*   **`=`, `<>`, `<`, `>`, `<=`, `>=`** — сравнение
*   **`BETWEEN a AND b`** — диапазон (включительно)
*   **`IN (val1, val2, ...)`** — значение в списке
*   **`LIKE 'pattern%'`** — поиск по шаблону (`%` любое количество, `_` один символ)
*   **`IS NULL`, `IS NOT NULL`** — проверка на NULL

#### **Логические операторы:**
*   **`AND`** — оба условия истинны
*   **`OR`** — хотя бы одно условие истинно
*   **`NOT`** — отрицание условия
*   **Приоритет:** `()` > `NOT` > `AND` > `OR`

#### **Функции работы с NULL:**
*   **`COALESCE(val1, val2, ...)`** — первое не-NULL значение
*   **`IFNULL(val, default)`** (MySQL) — заменяет NULL на default
*   **`NULLIF(val1, val2)`** — возвращает NULL если значения равны

#### **Важные замечания:**
*   Всегда используйте `ORDER BY` с `LIMIT`
*   Избегайте функций в `WHERE` на индексированных полях
*   Для проверки на NULL используйте `IS NULL`, а не `= NULL`
*   Выбирайте только нужные столбцы, а не `SELECT *`
*   Используйте `EXPLAIN` для анализа медленных запросов

## РАЗДЕЛ 3: Агрегация и группировка

### 3.1. Агрегатные функции (COUNT, SUM, AVG, MIN, MAX)

**Что такое агрегатные функции?**
Функции, которые выполняют вычисление на наборе строк и возвращают **одно** значение. Они "сворачивают" множество строк в одну.

**Основные агрегатные функции:**

| Функция | Описание | Особенности работы с NULL |
|---------|----------|---------------------------|
| `COUNT()` | Подсчет количества строк | `COUNT(*)` считает все строки, `COUNT(column)` игнорирует NULL |
| `SUM()` | Сумма значений | Игнорирует NULL значения |
| `AVG()` | Среднее значение | Игнорирует NULL, для пустого набора возвращает NULL |
| `MIN()` | Минимальное значение | Игнорирует NULL |
| `MAX()` | Максимальное значение | Игнорирует NULL |
| `GROUP_CONCAT()` (MySQL) / `STRING_AGG()` (PG) | Объединение строк | Конкатенирует значения в строку |

**Примеры использования:**
```sql
-- Базовая статистика по таблице
SELECT 
    COUNT(*) AS total_orders,
    SUM(amount) AS total_revenue,
    AVG(amount) AS avg_order_value,
    MIN(amount) AS min_order,
    MAX(amount) AS max_order
FROM orders;

-- COUNT с DISTINCT (уникальные значения)
SELECT 
    COUNT(DISTINCT customer_id) AS unique_customers,
    COUNT(DISTINCT product_id) AS unique_products
FROM orders;

-- Объединение значений в строку (MySQL)
SELECT 
    department,
    GROUP_CONCAT(DISTINCT name ORDER BY name SEPARATOR ', ') AS employees
FROM employees
GROUP BY department;

-- Объединение значений (PostgreSQL)
SELECT 
    department,
    STRING_AGG(DISTINCT name, ', ' ORDER BY name) AS employees
FROM employees
GROUP BY department;
```
#### **COUNT(*) vs COUNT(column)**

```sql
-- Таблица с данными:
-- id | name | phone
-- 1  | Иван | +7...
-- 2  | Анна | NULL
-- 3  | NULL | +7...

SELECT 
    COUNT(*) AS total_rows,           -- 3 (все строки)
    COUNT(name) AS names_count,       -- 2 (только не-NULL в name)
    COUNT(phone) AS phones_count,     -- 2 (только не-NULL в phone)
    COUNT(DISTINCT name) AS unique_names  -- 2 (Иван, Анна)
FROM users;
```

### **3.2. GROUP BY и HAVING**

#### **GROUP BY — группировка строк**
Группирует строки с одинаковыми значениями в указанных столбцах для применения агрегатных функций к каждой группе.

```sql
-- Группировка по одному столбцу
SELECT 
    department,
    COUNT(*) AS employee_count,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department;

-- Группировка по нескольким столбцам
SELECT 
    department,
    job_title,
    COUNT(*) AS count,
    SUM(salary) AS total_salary
FROM employees
GROUP BY department, job_title;

-- Группировка с выражениями
SELECT 
    EXTRACT(YEAR FROM hire_date) AS hire_year,
    COUNT(*) AS hires_count
FROM employees
GROUP BY EXTRACT(YEAR FROM hire_date)
ORDER BY hire_year;
```

#### **HAVING — фильтрация после группировки**
Фильтрует результаты агрегации (группы), тогда как WHERE фильтрует строки до группировки.

```sql
-- Найти отделы с более чем 10 сотрудниками
SELECT 
    department,
    COUNT(*) AS employee_count
FROM employees
GROUP BY department
HAVING COUNT(*) > 10;

-- Отделы со средней зарплатой выше 70000
SELECT 
    department,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department
HAVING AVG(salary) > 70000
ORDER BY avg_salary DESC;

-- Комбинирование WHERE и HAVING
SELECT 
    department,
    AVG(salary) AS avg_salary,
    COUNT(*) AS emp_count
FROM employees
WHERE hire_date >= '2020-01-01'   -- Фильтр строк ДО группировки
GROUP BY department
HAVING AVG(salary) > 50000        -- Фильтр групп ПОСЛЕ группировки
   AND COUNT(*) >= 5;             -- Можно использовать несколько условий
```

### **3.3. Фильтрация до и после агрегации**

#### **Порядок выполнения запроса с GROUP BY:**
1.  **`FROM`** — получение данных из таблицы
2.  **`WHERE`** — фильтрация строк (до группировки)
3.  **`GROUP BY`** — группировка строк
4.  **`HAVING`** — фильтрация групп (после группировки)
5.  **`SELECT`** — выбор столбцов и вычисление агрегатов
6.  **`ORDER BY`** — сортировка
7.  **`LIMIT`** — ограничение

```sql
-- Пример, показывающий разницу WHERE и HAVING
SELECT 
    department,
    COUNT(*) AS total,
    AVG(salary) AS avg_salary
FROM employees
WHERE salary > 30000           -- Исключаем низкооплачиваемых ДО подсчета среднего
GROUP BY department
HAVING AVG(salary) > 50000    -- Исключаем отделы с низкой средней ПОСЛЕ подсчета
ORDER BY avg_salary DESC;

-- Практический пример: анализ продаж
SELECT 
    p.category,
    COUNT(DISTINCT o.customer_id) AS unique_customers,
    SUM(o.quantity) AS total_quantity,
    SUM(o.quantity * p.price) AS total_revenue,
    AVG(o.quantity * p.price) AS avg_order_value
FROM orders o
JOIN products p ON o.product_id = p.id
WHERE o.order_date >= '2024-01-01'   -- Только заказы 2024 года
  AND o.status = 'completed'         -- Только завершенные
GROUP BY p.category
HAVING SUM(o.quantity * p.price) > 10000  -- Категории с выручкой > 10000
ORDER BY total_revenue DESC;
```

### **3.4. DISTINCT и GROUP BY для уникальных значений**

#### **DISTINCT vs GROUP BY:**

```sql
-- DISTINCT: только уникальные значения (без агрегации)
SELECT DISTINCT department FROM employees;

-- GROUP BY: уникальные значения + возможность агрегации
SELECT department FROM employees GROUP BY department;

-- Оба дают одинаковый результат для уникальности,
-- но GROUP BY более гибкий (можно добавить агрегатные функции)

-- Пример разницы:
-- DISTINCT с агрегацией (считает по всей таблице)
SELECT COUNT(DISTINCT department) AS dept_count FROM employees;

-- GROUP BY с агрегацией (считает по группам)
SELECT department, COUNT(*) FROM employees GROUP BY department;
```

#### **Удаление дубликатов:**

```sql
-- Найти дубликаты по email
SELECT email, COUNT(*) AS duplicate_count
FROM users
GROUP BY email
HAVING COUNT(*) > 1;

-- Удалить дубликаты, оставив самую новую запись
DELETE FROM users
WHERE id NOT IN (
    SELECT MIN(id)
    FROM users
    GROUP BY email
);

-- Альтернативный способ с временной таблицей
CREATE TABLE users_dedup AS
SELECT DISTINCT * FROM users;
```

### **Антипаттерны раздела 3**

#### ❌ **Смешивание агрегированных и неагрегированных столбцов без GROUP BY**

```sql
-- ❌ ПЛОХО: Ошибка в большинстве СУБД (или недетерминированный результат)
SELECT name, AVG(salary) FROM employees;

-- ✅ ХОРОШО: Добавить все неагрегированные столбцы в GROUP BY
SELECT department, AVG(salary) FROM employees GROUP BY department;

-- ИЛИ использовать оконные функции
SELECT name, salary, AVG(salary) OVER() FROM employees;
```

#### ❌ **Использование HAVING без GROUP BY для фильтрации агрегатов**

```sql
-- ❌ ПЛОХО: HAVING без GROUP BY работает, но это не интуитивно
SELECT AVG(salary) FROM employees HAVING AVG(salary) > 50000;

-- ✅ ХОРОШО: Использовать вложенный запрос
SELECT * FROM (
    SELECT AVG(salary) AS avg_salary FROM employees
) t WHERE avg_salary > 50000;
```

#### ❌ **Группировка по слишком большому числу столбцов**

```sql
-- ❌ ПЛОХО: Слишком детальная группировка, почти каждая строка — отдельная группа
SELECT first_name, last_name, email, phone, address, department, 
       COUNT(*) -- Всегда 1
FROM employees
GROUP BY first_name, last_name, email, phone, address, department;

-- ✅ ХОРОШО: Группировать только по значимым для анализа признакам
SELECT department, job_level, COUNT(*) AS employee_count
FROM employees
GROUP BY department, job_level;
```

#### ❌ **Неоптимальное использование DISTINCT**

```sql
-- ❌ ПЛОХО: DISTINCT на большом наборе данных
SELECT DISTINCT * FROM huge_log_table;

-- ✅ ХОРОШО: Сначала отфильтровать, затем применять DISTINCT
SELECT DISTINCT user_id, action 
FROM huge_log_table 
WHERE date >= '2024-01-01';

-- ИЛИ использовать GROUP BY если нужны агрегаты
SELECT user_id, action, COUNT(*) 
FROM huge_log_table 
GROUP BY user_id, action;
```

### **Лучшие практики раздела 3**

#### ✅ **Всегда включать неагрегированные столбцы в GROUP BY**

```sql
-- Правило: все столбцы в SELECT, не находящиеся внутри агрегатной функции,
-- должны быть в GROUP BY
SELECT 
    department,        -- в GROUP BY
    job_title,         -- в GROUP BY  
    COUNT(*) AS count,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department, job_title;  -- Все неагрегированные столбцы
```

#### ✅ **Фильтровать на ранних этапах с помощью WHERE**

```sql
-- Оптимизация: фильтровать до группировки, чтобы уменьшить объем данных
-- ❌ ПЛОХО: Сначала все сгруппировать, потом отфильтровать
SELECT category, AVG(price)
FROM products
GROUP BY category
HAVING AVG(price) > 100 AND category LIKE 'E%';

-- ✅ ХОРОШО: Отфильтровать строки до группировки
SELECT category, AVG(price)
FROM products
WHERE category LIKE 'E%'  -- Уменьшаем данные для группировки
GROUP BY category
HAVING AVG(price) > 100;
```

#### ✅ **Использовать составные агрегаты для комплексного анализа**

```sql
-- Анализ распределения в одном запросе
SELECT 
    department,
    COUNT(*) AS total,
    SUM(CASE WHEN salary > 70000 THEN 1 ELSE 0 END) AS high_earners,
    SUM(CASE WHEN salary < 40000 THEN 1 ELSE 0 END) AS low_earners,
    ROUND(AVG(salary), 2) AS avg_salary,
    MIN(salary) AS min_salary,
    MAX(salary) AS max_salary,
    ROUND(100.0 * SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) / COUNT(*), 2) AS female_percentage
FROM employees
GROUP BY department;
```

#### ✅ **Создавать индексы для полей в GROUP BY и WHERE**

```sql
-- Индексы для ускорения группировки
CREATE INDEX idx_employees_dept ON employees(department);
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
CREATE INDEX idx_products_category_price ON products(category, price);

-- Составные индексы для часто используемых комбинаций GROUP BY
CREATE INDEX idx_sales_analysis ON sales(region, product_category, sale_date);
```

### **Различия MySQL и PostgreSQL**

| Функциональность | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Конкатенация строк в группе** | `GROUP_CONCAT(col SEPARATOR ', ')` | `STRING_AGG(col, ', ')` |
| **Фильтр в агрегатной функции** | Не поддерживается `FILTER` | `SUM(col) FILTER (WHERE condition)` |
| **Статистические агрегаты** | `STDDEV_POP()`, `VAR_POP()` | `STDDEV()`, `VARIANCE()` |
| **Перцентили** | `PERCENTILE_CONT()` не поддерживается | `PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY col)` |
| **Режим ONLY_FULL_GROUP_BY** | По умолчанию строгий с версии 5.7 | Всегда строгий |
| **Группировка по номеру столбца** | `GROUP BY 1, 2` | Не рекомендуется, лучше по имени |

#### **Пример FILTER в PostgreSQL:**

```sql
-- Агрегация с условием для каждой функции
SELECT 
    department,
    COUNT(*) AS total,
    COUNT(*) FILTER (WHERE salary > 70000) AS high_paid,
    AVG(salary) FILTER (WHERE hire_date > '2020-01-01') AS avg_newhire_salary
FROM employees
GROUP BY department;
```

### **Шпаргалка раздела 3**

#### **Базовый синтаксис GROUP BY:**

```sql
SELECT column1, AGG_FUNC(column2)
FROM table
WHERE condition
GROUP BY column1
HAVING condition_for_groups
ORDER BY column1
LIMIT N;
```

#### **Основные агрегатные функции:**
*   **`COUNT(*)`** — количество всех строк
*   **`COUNT(column)`** — количество не-NULL значений
*   **`SUM(column)`** — сумма значений
*   **`AVG(column)`** — среднее значение
*   **`MIN(column)`**, **`MAX(column)`** — минимальное/максимальное
*   **`GROUP_CONCAT()`** / **`STRING_AGG()`** — объединение строк

#### **Порядок выполнения:**
1.  **`FROM`** → 2. **`WHERE`** → 3. **`GROUP BY`** → 4. **`HAVING`** → 5. **`SELECT`** → 6. **`ORDER BY`** → 7. **`LIMIT`**

#### **Ключевые отличия WHERE vs HAVING:**
*   **`WHERE`** — фильтрация строк **до** группировки
*   **`HAVING`** — фильтрация групп **после** группировки
*   **`WHERE`** **не может** использовать агрегатные функции
*   **`HAVING`** **может** использовать агрегатные функции

#### **Типичные задачи и решения:**

```sql
-- 1. Подсчет уникальных значений
SELECT COUNT(DISTINCT column) FROM table;

-- 2. Нахождение дубликатов  
SELECT column, COUNT(*) FROM table GROUP BY column HAVING COUNT(*) > 1;

-- 3. Группировка по диапазонам
SELECT 
    CASE 
        WHEN salary < 40000 THEN 'Low'
        WHEN salary BETWEEN 40000 AND 80000 THEN 'Medium'
        ELSE 'High'
    END AS salary_group,
    COUNT(*) AS count
FROM employees
GROUP BY salary_group;

-- 4. Агрегация по периодам времени
SELECT 
    DATE_TRUNC('month', order_date) AS month,
    COUNT(*) AS orders_count,
    SUM(amount) AS total_amount
FROM orders
GROUP BY DATE_TRUNC('month', order_date)
ORDER BY month;
```

#### **Важные замечания:**
*   Все неагрегированные столбцы в `SELECT` должны быть в `GROUP BY`
*   Используйте `WHERE` для фильтрации строк, `HAVING` — для фильтрации групп
*   `COUNT(*)` быстрее `COUNT(column)` — не проверяет NULL
*   Для удаления дубликатов `GROUP BY` обычно эффективнее `DISTINCT`
*   Создавайте индексы на полях, используемых в `GROUP BY` и `WHERE`

## РАЗДЕЛ 4: СОЕДИНЕНИЯ ТАБЛИЦ (JOINs)

### 4.1. INNER JOIN

**Что такое INNER JOIN?**
INNER JOIN возвращает только те строки, для которых нашлось соответствие **в обеих** соединяемых таблицах. Это самый распространенный тип соединения.

**Синтаксис:**
```sql
SELECT столбцы
FROM таблица1
INNER JOIN таблица2
    ON таблица1.столбец = таблица2.столбец;
-- Ключевое слово INNER часто опускают
SELECT столбцы FROM таблица1 JOIN таблица2 ON условие;
```

**Пример с данными:**
```sql
-- Таблицы для примеров
CREATE TABLE departments (
    id INT PRIMARY KEY,
    name VARCHAR(50)
);
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    department_id INT,
    salary DECIMAL(10,2),
    FOREIGN KEY (department_id) REFERENCES departments(id)
);

-- INNER JOIN: сотрудники с привязанными отделами
SELECT
    e.name AS employee_name,
    e.salary,
    d.name AS department_name
FROM employees e
INNER JOIN departments d ON e.department_id = d.id;
```

**Особенности INNER JOIN:**
- Порядок таблиц не важен для результата (но может влиять на производительность)
- Если условие JOIN не выполняется, строка исключается из результата
- Можно соединять несколько таблиц в цепочку

### 4.2. LEFT JOIN и RIGHT JOIN

**LEFT (OUTER) JOIN**
Возвращает все строки из левой таблицы, даже если для них нет соответствия в правой таблице. Для отсутствующих соответствий возвращаются NULL.
```sql
-- Все сотрудники, даже без отделов
SELECT
    e.name AS employee_name,
    e.salary,
    d.name AS department_name
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id;
```

**RIGHT (OUTER) JOIN**
Возвращает все строки из правой таблицы, даже если для них нет соответствия в левой. Используется реже, так как обычно можно поменять таблицы местами и использовать LEFT JOIN.
```sql
-- Все отделы, даже без сотрудников
SELECT
    d.name AS department_name,
    e.name AS employee_name
FROM employees e
RIGHT JOIN departments d ON e.department_id = d.id;

-- Аналогично с LEFT JOIN (более читаемо):
SELECT
    d.name AS department_name,
    e.name AS employee_name
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id;
```

**Практическое использование LEFT JOIN:**
```sql
-- 1. Найти сотрудников без отдела
SELECT e.*
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id
WHERE d.id IS NULL;

-- 2. Найти отделы без сотрудников
SELECT d.*
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id
WHERE e.id IS NULL;
```

### 4.3. FULL OUTER JOIN

**Что такое FULL OUTER JOIN?**
Возвращает все строки из обеих таблиц, объединяя их там, где есть соответствие. Если соответствия нет, для недостающих столбцов возвращается NULL.
```sql
-- Все сотрудники и все отделы
SELECT
    e.name AS employee_name,
    d.name AS department_name
FROM employees e
FULL OUTER JOIN departments d ON e.department_id = d.id;
```

**Эмуляция FULL JOIN в MySQL:**
MySQL не поддерживает FULL OUTER JOIN напрямую, но его можно эмулировать:
```sql
SELECT
    e.name AS employee_name,
    d.name AS department_name
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id
UNION ALL
SELECT
    e.name AS employee_name,
    d.name AS department_name
FROM employees e
RIGHT JOIN departments d ON e.department_id = d.id
WHERE e.department_id IS NULL;
```

### 4.4. CROSS JOIN

**Что такое CROSS JOIN?**
Декартово произведение: каждая строка первой таблицы соединяется с каждой строкой второй таблицы.
```sql
-- Все комбинации размеров и цветов
SELECT
    s.size_name,
    c.color_name
FROM sizes s
CROSS JOIN colors c;

-- То же самое через запятую (неявный CROSS JOIN)
SELECT s.size_name, c.color_name
FROM sizes s, colors c;
```

**Важные моменты:**
- CROSS JOIN не имеет условия ON
- Может генерировать огромное количество строк
- Полезен для создания тестовых данных или комбинаций

### 4.5. SELF JOIN

Соединение таблицы с самой собой для сравнения строк внутри одной таблицы.
```sql
-- Таблица сотрудников с менеджерами
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    manager_id INT
);

-- Найти сотрудников и их менеджеров
SELECT
    e.name AS employee_name,
    m.name AS manager_name
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.id; -- SELF JOIN

-- Найти сотрудников с зарплатой выше их менеджера
SELECT
    e.name AS employee_name,
    e.salary,
    m.name AS manager_name,
    m.salary AS manager_salary
FROM employees e
JOIN employees m ON e.manager_id = m.id
WHERE e.salary > m.salary;
```

### 4.6. NATURAL JOIN и USING

**NATURAL JOIN**
Автоматически соединяет таблицы по одинаковым именам столбцов. Опасен и не рекомендуется к использованию.
```sql
SELECT *
FROM employees
NATURAL JOIN departments;
```

**USING для простых случаев**
Если столбцы соединения имеют одинаковое имя:
```sql
SELECT *
FROM employees
JOIN departments USING (department_id);
```

### 4.7. Неявные vs явные JOIN

**Неявный JOIN (устаревший стиль):**
```sql
SELECT e.name, d.name
FROM employees e, departments d
WHERE e.department_id = d.id;
```

**Явный JOIN (современный стиль):**
```sql
SELECT e.name, d.name
FROM employees e
JOIN departments d ON e.department_id = d.id;
```

**Почему явный JOIN лучше:**
- Читаемость: условие соединения отделено от условий фильтрации
- Безопасность: меньше шансов забыть условие соединения
- Поддержка OUTER JOIN

**Антипаттерны раздела 4**

❌ Забытое условие JOIN (случайный CROSS JOIN)
```sql
-- ПЛОХО: Нет условия ON -> CROSS JOIN
SELECT * FROM employees JOIN departments;
```

✅ ХОРОШО
```sql
SELECT * FROM employees e
JOIN departments d ON e.department_id = d.id;
```

❌ Использование SELECT * в JOIN
```sql
-- ПЛОХО: Дублирующиеся имена столбцов
SELECT * FROM employees e JOIN departments d ON e.department_id = d.id;
```

✅ ХОРОШО
```sql
SELECT
    e.id AS employee_id,
    e.name AS employee_name,
    d.id AS department_id,
    d.name AS department_name
FROM employees e
JOIN departments d ON e.department_id = d.id;
```

❌ Неправильный тип JOIN
```sql
-- ПЛОХО: INNER JOIN когда нужны все строки из одной таблицы
SELECT e.name, d.name
FROM employees e
JOIN departments d ON e.department_id = d.id;
```

✅ ХОРОШО
```sql
SELECT e.name, d.name
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id;
```

**Лучшие практики раздела 4**

✅ Использовать алиасы (псевдонимы) для таблиц
```sql
SELECT
    e.name AS emp_name,
    d.name AS dept_name
FROM employees e
JOIN departments d ON e.department_id = d.id;
```

✅ Порядок таблиц в JOIN имеет значение для производительности
```sql
SELECT * FROM small_table s
JOIN large_table l ON s.id = l.small_id; -- Лучше начинать с маленькой таблицы
```

✅ Создавать индексы на полях соединения
```sql
CREATE INDEX idx_employees_department ON employees(department_id);
```

✅ Использовать EXPLAIN для анализа сложных JOIN
```sql
EXPLAIN
SELECT e.name, d.name
FROM employees e
JOIN departments d ON e.department_id = d.id;
```

**Различия MySQL и PostgreSQL**

| Особенность | MySQL | PostgreSQL |
|-------------|-------|------------|
| FULL OUTER JOIN | Не поддерживается (эмулировать через UNION) | Полная поддержка |
| NATURAL JOIN | Поддерживается (но опасен) | Поддерживается (но опасен) |
| USING синтаксис | Поддерживается | Поддерживается |

**Шпаргалка раздела 4**

**Синтаксис JOIN:**
```sql
SELECT столбцы
FROM таблица1
[INNER|LEFT|RIGHT|FULL] JOIN таблица2
    ON условие_соединения
[WHERE условия_фильтрации]
[ORDER BY сортировка]
```

**Типы JOIN и их результаты:**

| Тип JOIN | Возвращает | Когда использовать |
|----------|------------|--------------------|
| INNER JOIN | Только совпадающие строки | Связь обязательна (заказы-товары) |
| LEFT JOIN | Все строки слева + совпадения справа | Главная таблица + опциональные связи |
| RIGHT JOIN | Все строки справа + совпадения слева | Редко, лучше поменять таблицы |
| FULL JOIN | Все строки из обеих таблиц | Когда нужны все данные из обеих таблиц |
| CROSS JOIN | Все комбинации | Комбинации (размеры-цветы), тестовые данные |

**Ключевые правила:**
- Всегда указывайте условие JOIN (кроме CROSS JOIN)
- Используйте алиасы для удобства чтения
- Для опциональных связей используйте LEFT JOIN
- Фильтруйте данные до JOIN, когда это возможно
- Создавайте индексы на полях соединения
- Избегайте NATURAL JOIN (опасно)
- Для иерархических данных используйте SELF JOIN

**Примеры для собеседований:**
```sql
-- Q: Как найти клиентов без заказов?
SELECT c.*
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id
WHERE o.id IS NULL;

-- Q: Как найти менеджера с наибольшим количеством подчиненных?
SELECT m.name, COUNT(e.id) AS subordinates
FROM employees e
JOIN employees m ON e.manager_id = m.id
GROUP BY m.id, m.name
ORDER BY subordinates DESC
LIMIT 1;
```

## РАЗДЕЛ 5: ПОДЗАПРОСЫ (Subqueries)

### 5.1. Типы подзапросов

1. Скалярный (возвращает одно значение)
```sql
SELECT name, salary
FROM employees
WHERE salary > (SELECT AVG(salary) FROM employees);
```

2. Столбцовый (возвращает один столбец)
```sql
SELECT name
FROM employees
WHERE department_id IN (
    SELECT id FROM departments WHERE location = 'Moscow'
);
```

3. Строковый (возвращает одну строку)
```sql
SELECT *
FROM employees
WHERE (name, salary) = (
    SELECT name, salary FROM employees ORDER BY salary DESC LIMIT 1
);
```

4. Табличный (возвращает таблицу)
```sql
SELECT *
FROM (
    SELECT department_id, AVG(salary) avg_sal
    FROM employees
    GROUP BY department_id
) dept_avg
WHERE avg_sal > 60000;
```

### 5.2. Коррелированные подзапросы
Выполняются для каждой строки внешнего запроса.

```sql
-- Сотрудники, зарабатывающие больше среднего в своём отделе
SELECT name, salary, department_id
FROM employees e
WHERE salary > (
    SELECT AVG(salary)
    FROM employees
    WHERE department_id = e.department_id
);
```

```sql
-- Заказы с суммой выше среднего по клиенту
SELECT o.id, o.customer_id, o.amount
FROM orders o
WHERE o.amount > (
    SELECT AVG(amount)
    FROM orders
    WHERE customer_id = o.customer_id
);
```

### 5.3. EXISTS / NOT EXISTS

```sql
-- Клиенты, сделавшие хотя бы один заказ
SELECT name, email
FROM customers c
WHERE EXISTS (
    SELECT 1 FROM orders o WHERE o.customer_id = c.id
);
```

```sql
-- Клиенты без заказов (часто быстрее NOT IN)
SELECT name, email
FROM customers c
WHERE NOT EXISTS (
    SELECT 1 FROM orders o WHERE o.customer_id = c.id
);
```

### 5.4. Подзапросы в SELECT, FROM, WHERE

```sql
-- В SELECT: вычисляемое поле
SELECT
    name,
    salary,
    (SELECT AVG(salary) FROM employees) AS company_avg
FROM employees;
```

```sql
-- В FROM: производная таблица
SELECT d.name, high.avg_salary
FROM departments d
JOIN (
    SELECT department_id, AVG(salary) avg_salary
    FROM employees
    GROUP BY department_id
    HAVING AVG(salary) > 70000
) high ON d.id = high.department_id;
```

**Антипаттерны раздела 5**

❌ NOT IN с возможными NULL
```sql
-- ПЛОХО: если в подзапросе есть NULL → весь результат пустой
SELECT * FROM customers
WHERE id NOT IN (SELECT customer_id FROM orders);
```

✅ ХОРОШО
```sql
SELECT * FROM customers c
WHERE NOT EXISTS (SELECT 1 FROM orders o WHERE o.customer_id = c.id);
```

❌ Коррелированный подзапрос вместо JOIN
```sql
-- Медленно
SELECT name FROM employees e
WHERE salary > (SELECT AVG(salary) FROM employees WHERE dept = e.dept);
```

✅ ХОРОШО
```sql
WITH dept_avg AS (
    SELECT department_id, AVG(salary) avg_sal
    FROM employees GROUP BY department_id
)
SELECT e.name
FROM employees e
JOIN dept_avg da ON e.department_id = da.department_id
WHERE e.salary > da.avg_sal;
```

**Лучшие практики раздела 5**

- Предпочитай JOIN / CTE коррелированным подзапросам
- Используй EXISTS вместо IN для больших наборов
- Избегай NOT IN с колонками, где могут быть NULL
- Для скалярных подзапросов — лучше CTE или JOIN

**Шпаргалка раздела 5**
```sql
WHERE col > (SELECT AVG(col) FROM table)
WHERE EXISTS (SELECT 1 FROM t2 WHERE t2.fk = t1.id)
FROM (SELECT ... GROUP BY ...) AS sub
```

## РАЗДЕЛ 6: CTE (Common Table Expressions)

### 6.1. Простые CTE

CTE — временная именованная таблица, которая улучшает читаемость и позволяет переиспользовать подзапросы.

```sql
WITH monthly_sales AS (
    SELECT DATE_TRUNC('month', sale_date) AS month,
           SUM(amount) AS total
    FROM sales
    GROUP BY 1
)
SELECT month,
       total,
       total * 1.1 AS total_with_tax
FROM monthly_sales
ORDER BY month;
```

### 6.2. Несколько CTE в одном запросе

```sql
WITH active_users AS (
    SELECT id, name
    FROM users
    WHERE last_login >= CURRENT_DATE - INTERVAL '30 days'
),
user_orders AS (
    SELECT u.id,
           u.name,
           COUNT(o.id) AS order_count,
           SUM(o.amount) AS total_spent
    FROM active_users u
    LEFT JOIN orders o ON o.user_id = u.id
    GROUP BY u.id, u.name
)
SELECT *
FROM user_orders
WHERE order_count > 0
ORDER BY total_spent DESC;
```

### 6.3. Рекурсивные CTE

Идеальны для иерархий, деревьев, графов, генерации последовательностей.

```sql
WITH RECURSIVE hierarchy AS (
    -- Якорная часть
    SELECT id,
           name,
           manager_id,
           1 AS level,
           name AS path
    FROM employees
    WHERE manager_id IS NULL

    UNION ALL

    -- Рекурсивная часть
    SELECT e.id,
           e.name,
           e.manager_id,
           h.level + 1,
           h.path || ' → ' || e.name
    FROM employees e
    INNER JOIN hierarchy h ON e.manager_id = h.id
)
SELECT id,
       name,
       level,
       REPEAT('  ', level-1) || name AS tree_view,
       path
FROM hierarchy
ORDER BY path;
```

Генерация последовательности дат:
```sql
WITH RECURSIVE dates AS (
    SELECT '2025-01-01'::date AS dt
    UNION ALL
    SELECT dt + INTERVAL '1 day'
    FROM dates
    WHERE dt < '2025-12-31'
)
SELECT dt FROM dates;
```

### 6.4. CTE vs Подзапросы vs Временные таблицы

| Сценарий              | Подзапрос | CTE       | Временная таблица |
|-----------------------|-----------|-----------|-------------------|
| Читаемость            | Средне    | Высокая   | Высокая           |
| Переиспользование     | Нет       | Да        | Да                |
| Рекурсия              | Нет       | Да        | Нет               |
| Производительность    | Лучше     | Почти то же| Лучше на больших  |

**Антипаттерны раздела 6**

❌ Рекурсия без ограничения глубины
```sql
WITH RECURSIVE bad AS (
    SELECT ... UNION ALL SELECT ... FROM bad
)
```
→ Бесконечный цикл при циклических ссылках

✅ ХОРОШО
```sql
WHERE level < 100
```

**Лучшие практики раздела 6**

- Используй CTE для сложных запросов с повторяющимися подзапросами
- Именуй CTE осмысленно (не t1, t2)
- В рекурсивных CTE всегда добавляй ограничение глубины
- Для очень больших данных иногда лучше материализовать в temp table

**Шпаргалка раздела 6**
```sql
-- Простой CTE
WITH cte AS (SELECT ...)
SELECT * FROM cte;

-- Рекурсивный
WITH RECURSIVE cte AS (
    SELECT ... -- anchor
    UNION ALL
    SELECT ... FROM cte WHERE ...
)
SELECT * FROM cte;

-- С ограничением
WHERE level < 20
```

## РАЗДЕЛ 7: ОКОННЫЕ ФУНКЦИИ

### 7.1. Основы оконных функций

Оконные функции выполняют расчёты по набору строк (окну), связанных с текущей строкой, без группировки данных.

```sql
SELECT 
    department,
    name,
    salary,
    SUM(salary) OVER (PARTITION BY department) AS dept_total,
    AVG(salary) OVER (PARTITION BY department) AS dept_avg,
    salary / SUM(salary) OVER (PARTITION BY department) * 100 AS pct_of_dept
FROM employees;
```

**PARTITION BY** — группирует строки внутри окна (как GROUP BY, но сохраняет все строки).
**ORDER BY** внутри OVER() — определяет порядок внутри окна (важно для кумулятивных функций).

### 7.2. Ранжирование

```sql
SELECT 
    department,
    name,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS rn,
    RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS rank,
    DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dense_rank,
    NTILE(4) OVER (PARTITION BY department ORDER BY salary DESC) AS quartile
FROM employees;
```

**Разница:**
- ROW_NUMBER() — уникальный номер (1,2,3,4...)
- RANK() — ранг с пропусками (1,2,2,4...)
- DENSE_RANK() — ранг без пропусков (1,2,2,3...)
- NTILE(n) — делит на n групп

**Топ-3 в группе**
```sql
WITH ranked AS (
    SELECT 
        category,
        product_name,
        price,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) AS rn
    FROM products
)
SELECT * FROM ranked WHERE rn <= 3;
```

### 7.3. Кумулятивные и скользящие метрики

**Кумулятивная сумма**
```sql
SELECT 
    sale_date,
    amount,
    SUM(amount) OVER (ORDER BY sale_date) AS running_total,
    SUM(amount) OVER (PARTITION BY EXTRACT(YEAR FROM sale_date) ORDER BY sale_date) AS ytd_total
FROM sales
ORDER BY sale_date;
```

**Скользящее среднее за 7 дней**
```sql
SELECT 
    sale_date,
    revenue,
    AVG(revenue) OVER (
        ORDER BY sale_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS ma_7d
FROM daily_revenue
ORDER BY sale_date;
```

### 7.4. Смещение строк (LAG / LEAD)

```sql
SELECT 
    sale_date,
    amount,
    LAG(amount, 1) OVER (ORDER BY sale_date) AS prev_day,
    LEAD(amount, 1) OVER (ORDER BY sale_date) AS next_day,
    amount - LAG(amount, 1) OVER (ORDER BY sale_date) AS day_change,
    (amount - LAG(amount, 1) OVER (ORDER BY sale_date)) / LAG(amount, 1) OVER (ORDER BY sale_date) * 100 AS pct_change
FROM sales;
```

### 7.5. FIRST_VALUE / LAST_VALUE / NTH_VALUE

```sql
SELECT 
    department,
    name,
    salary,
    FIRST_VALUE(name) OVER (PARTITION BY department ORDER BY salary DESC) AS top_earner,
    LAST_VALUE(name) OVER (
        PARTITION BY department
        ORDER BY salary DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS lowest_earner,
    NTH_VALUE(name, 2) OVER (PARTITION BY department ORDER BY salary DESC) AS second_highest
FROM employees;
```

**Антипаттерны раздела 7**

❌ Оконные функции в WHERE (нельзя напрямую)
```sql
-- ПЛОХО — ошибка
WHERE ROW_NUMBER() OVER (...) = 1
```

✅ ХОРОШО
```sql
WITH ranked AS (
    SELECT *, ROW_NUMBER() OVER (...) AS rn
    FROM ...
)
SELECT * FROM ranked WHERE rn = 1;
```

❌ LAST_VALUE без полного фрейма
```sql
-- ПЛОХО — возвращает текущую строку
LAST_VALUE(value) OVER (ORDER BY date)
```

✅ ХОРОШО
```sql
LAST_VALUE(value) OVER (
    ORDER BY date
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)
```

**Лучшие практики раздела 7**

- Используй PARTITION BY для группировки внутри окна
- ORDER BY внутри OVER() для кумулятивных расчётов
- ROWS BETWEEN / RANGE BETWEEN — контролируй границы окна
- Для топ-N всегда используй ROW_NUMBER() или RANK() + фильтр в CTE
- Избегай оконных функций на очень больших таблицах без индексов

**Шпаргалка раздела 7**
```sql
ROW_NUMBER() OVER (PARTITION BY col ORDER BY col DESC) rn
SUM(val) OVER (ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
LAG(val, 1) OVER (PARTITION BY group ORDER BY date)
AVG(val) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
FIRST_VALUE(val) OVER (PARTITION BY group ORDER BY date)
```

## РАЗДЕЛ 8: ОПТИМИЗАЦИЯ И ИНДЕКСЫ

### 8.1. Типы индексов

- **B-tree** — универсальный, по умолчанию в PostgreSQL и MySQL
- **Hash** — для точного равенства (=), быстрее B-tree на равенстве (только PostgreSQL)
- **GIN** — для массивов, JSONB, полнотекстового поиска (PostgreSQL)
- **GiST** — для геоданных, полнотекста, расстояний
- **BRIN** — для очень больших таблиц с естественным порядком (маленький размер индекса)

```sql
-- B-tree
CREATE INDEX idx_orders_date ON orders(order_date);

-- Hash (PostgreSQL)
CREATE INDEX idx_users_email_hash ON users USING HASH (email);

-- GIN для JSONB
CREATE INDEX idx_products_tags_gin ON products USING GIN (tags);

-- Покрывающий индекс (INCLUDE)
CREATE INDEX idx_orders_cover ON orders(customer_id) INCLUDE (amount, status);
```

### 8.2. Когда индекс используется

Индекс помогает при:
- WHERE col = value
- WHERE col > value / < value / BETWEEN
- ORDER BY col
- JOIN ON col
- GROUP BY col (иногда)

### 8.3. Когда индекс НЕ используется

❌ Функция над полем в WHERE
```sql
-- ПЛОХО — индекс не работает
WHERE YEAR(order_date) = 2024
WHERE UPPER(name) = 'SERGEY'
```

✅ ХОРОШО
```sql
WHERE order_date >= '2024-01-01' AND order_date < '2025-01-01'
WHERE name = 'Sergey' COLLATE "C"  -- если нужно регистронезависимо
```

❌ LIKE '%text%'
```sql
-- ПЛОХО — индекс не работает (кроме триграммного GIN)
WHERE email LIKE '%@gmail.com'
```

✅
```sql
WHERE email LIKE 'sergey%'
```

❌ Неселективный фильтр (мало уникальных значений)
```sql
-- ПЛОХО — сканирует почти всю таблицу
WHERE is_active = true
```

### 8.4. EXPLAIN и анализ

```sql
EXPLAIN ANALYZE
SELECT * FROM orders
WHERE order_date > '2024-01-01'
ORDER BY order_date;
```

Ключевые строки:
- **Index Scan** — хорошо
- **Seq Scan** — плохо (полное сканирование таблицы)
- **Bitmap Heap Scan** — средне
- **Cost** — оценка затрат
- **Actual time** / **Rows** — реальное время и строк

### 8.5. Лучшие практики индексации

- Индексируй столбцы в WHERE, JOIN, ORDER BY, GROUP BY
- Составные индексы (leftmost prefix rule)
```sql
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
```
- Покрывающие индексы (INCLUDE в PostgreSQL)
- Удаляй неиспользуемые индексы (pg_stat_user_indexes)
- Регулярно ANALYZE / VACUUM

**Антипаттерны раздела 8**

❌ Индекс на низкоселективном поле
```sql
CREATE INDEX idx_gender ON users(gender); -- всего 2 значения
```

❌ Индекс на каждое поле
→ Раздувание таблицы, замедление INSERT/UPDATE

**Шпаргалка раздела 8**
```sql
EXPLAIN ANALYZE SELECT ...
CREATE INDEX idx_name ON table(col1, col2) INCLUDE (col3);
CREATE INDEX idx_json ON table USING GIN (json_col);
```

## РАЗДЕЛ 9: ТРАНЗАКЦИИ И УПРАВЛЕНИЕ

### 9.1. Что такое транзакция (ACID)

- **Atomicity** (атомарность) — всё или ничего
- **Consistency** (согласованность) — данные остаются валидными
- **Isolation** (изоляция) — транзакции не видят промежуточных результатов друг друга
- **Durability** (долговечность) — после COMMIT изменения сохраняются даже при сбое

### 9.2. Управление транзакциями

```sql
BEGIN;  -- или BEGIN TRANSACTION;

UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;

COMMIT;  -- фиксируем изменения
-- или
ROLLBACK; -- откатываем всё
```

### 9.3. Savepoints

```sql
BEGIN;

UPDATE accounts SET balance = balance - 100 WHERE id = 1;
SAVEPOINT first_transfer;

UPDATE accounts SET balance = balance - 50 WHERE id = 1;
-- ошибка, откатываем только второе действие
ROLLBACK TO SAVEPOINT first_transfer;

COMMIT; -- фиксируем только первое действие
```

### 9.4. Уровни изоляции транзакций

| Уровень изоляции      | Dirty Read | Non-repeatable Read | Phantom Read | PostgreSQL default | MySQL default (InnoDB) |
|-----------------------|------------|---------------------|--------------|--------------------|------------------------|
| READ UNCOMMITTED      | Да         | Да                  | Да           | Нет                | Нет                    |
| READ COMMITTED        | Нет        | Да                  | Да           | Да                 | Нет                    |
| REPEATABLE READ       | Нет        | Нет                 | Да           | Нет                | Да                     |
| SERIALIZABLE          | Нет        | Нет                 | Нет          | Нет                | Нет                    |

```sql
-- PostgreSQL: установить уровень
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;

-- MySQL
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
```

### 9.5. Проблемы параллельного выполнения

- **Dirty Read** — чтение незафиксированных изменений
- **Non-repeatable Read** — повторное чтение строки даёт разные результаты
- **Phantom Read** — повторный SELECT возвращает новые строки
- **Lost Update** — два UPDATE теряют изменения друг друга

### 9.6. Блокировки и deadlocks

```sql
-- Явная блокировка строки
SELECT * FROM accounts WHERE id = 1 FOR UPDATE;

-- Проверка блокировок в PostgreSQL
SELECT * FROM pg_locks WHERE relation = 'accounts'::regclass;
```

**Deadlock** — две транзакции ждут друг друга.
PostgreSQL сам детектирует и откатывает одну из них.

### 9.7. Лучшие практики

- Держи транзакции короткими
- Начинай с BEGIN только когда нужно
- Используй SAVEPOINT для частичного отката
- Избегай SERIALIZABLE без необходимости (медленнее)
- Для аналитики используй READ COMMITTED + READ ONLY
```sql
BEGIN READ ONLY;
``` 

**Шпаргалка раздела 9**
```sql
BEGIN;
...
SAVEPOINT sp1;
...
ROLLBACK TO SAVEPOINT sp1;
COMMIT;

SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
SELECT ... FOR UPDATE;
```

## РАЗДЕЛ 10: РАБОТА С ДАТАМИ И СТРОКАМИ

### 10.1. Основные типы дат и времени

| Тип          | PostgreSQL          | MySQL               | Описание                          |
|--------------|---------------------|---------------------|-----------------------------------|
| Дата         | DATE                | DATE                | Только дата (2024-01-15)          |
| Время        | TIME                | TIME                | Только время (14:30:00)           |
| Дата+Время   | TIMESTAMP           | DATETIME            | Дата и время без часового пояса   |
| С часовым поясом | TIMESTAMPTZ     | TIMESTAMP           | Дата и время с часовым поясом     |
| Интервал     | INTERVAL            | Нет прямого аналога | Разница между датами              |

### 10.2. Текущая дата и время

```sql
-- PostgreSQL
SELECT CURRENT_DATE;          -- 2024-01-15
SELECT CURRENT_TIMESTAMP;     -- 2024-01-15 14:30:00.123456+00
SELECT NOW();                 -- синоним CURRENT_TIMESTAMP
SELECT CURRENT_TIME;          -- 14:30:00

-- MySQL
SELECT CURDATE();             -- 2024-01-15
SELECT NOW();                 -- 2024-01-15 14:30:00
SELECT CURRENT_TIME();        -- 14:30:00
```

### 10.3. Извлечение частей даты

```sql
-- PostgreSQL
SELECT EXTRACT(YEAR FROM ts) AS year,
       EXTRACT(MONTH FROM ts) AS month,
       EXTRACT(DAY FROM ts) AS day,
       EXTRACT(HOUR FROM ts) AS hour,
       DATE_PART('quarter', ts) AS quarter
FROM (SELECT '2024-03-15 14:30:00'::timestamp AS ts) t;

-- MySQL
SELECT YEAR(ts) AS year,
       MONTH(ts) AS month,
       DAY(ts) AS day,
       HOUR(ts) AS hour,
       QUARTER(ts) AS quarter
FROM (SELECT '2024-03-15 14:30:00' AS ts) t;
```

### 10.4. Форматирование дат и строк

```sql
-- PostgreSQL
SELECT TO_CHAR(CURRENT_DATE, 'YYYY-MM-DD') AS iso_date,
       TO_CHAR(CURRENT_TIMESTAMP, 'DD Mon YYYY HH24:MI:SS') AS readable,
       TO_CHAR(ts, 'DD.MM.YYYY') AS russian_format
FROM (SELECT NOW() AS ts) t;

-- MySQL
SELECT DATE_FORMAT(NOW(), '%Y-%m-%d') AS iso_date,
       DATE_FORMAT(NOW(), '%d %b %Y %H:%i:%s') AS readable,
       DATE_FORMAT(NOW(), '%d.%m.%Y') AS russian_format;
```

### 10.5. Операции с датами (интервалы)

```sql
-- PostgreSQL
SELECT CURRENT_DATE + INTERVAL '30 days' AS plus_30,
       CURRENT_DATE - INTERVAL '1 month' AS minus_month,
       age('2024-01-15', '2023-06-01') AS age_interval,
       '2024-01-15'::date - '2023-12-01'::date AS days_diff;

-- MySQL
SELECT DATE_ADD(CURDATE(), INTERVAL 30 DAY) AS plus_30,
       DATE_SUB(CURDATE(), INTERVAL 1 MONTH) AS minus_month,
       DATEDIFF('2024-01-15', '2023-06-01') AS days_diff;
```

### 10.6. Работа со строками

```sql
-- Конкатенация
-- PostgreSQL
SELECT 'Hello' || ' ' || 'World';
-- MySQL
SELECT CONCAT('Hello', ' ', 'World');

-- Подстрока
SELECT SUBSTRING('PostgreSQL', 1, 4);  -- Post
SELECT LEFT('PostgreSQL', 4);          -- Post
SELECT RIGHT('PostgreSQL', 3);         -- SQL

-- Поиск позиции
SELECT POSITION('SQL' IN 'PostgreSQL');  -- 6
SELECT LOCATE('SQL', 'PostgreSQL');      -- 6 (MySQL)

-- Замена
SELECT REPLACE('PostgreSQL is great', 'great', 'awesome');

-- Приведение регистра
SELECT UPPER('hello'), LOWER('WORLD');
SELECT INITCAP('hello world');  -- Hello World (PostgreSQL)
SELECT CONCAT(UPPER(LEFT('hello world', 1)), SUBSTRING('hello world' FROM 2));  -- MySQL аналог
```

**Шпаргалка раздела 10**
```sql
-- PostgreSQL
DATE_TRUNC('month', ts)          -- начало месяца
EXTRACT(YEAR FROM ts)            -- год
TO_CHAR(ts, 'YYYY-MM-DD HH24:MI') -- форматирование
ts + INTERVAL '1 day'            -- прибавить день
age(ts1, ts2)                    -- разница

-- MySQL
DATE_FORMAT(ts, '%Y-%m-%d')      -- форматирование
DATE_ADD(ts, INTERVAL 1 DAY)     -- прибавить день
DATEDIFF(ts1, ts2)               -- разница в днях
CONCAT(str1, str2)               -- конкатенация
```

## РАЗДЕЛ 11: ПРОДВИНУТЫЕ ТЕМЫ

### 11.1. JSON и JSONB в PostgreSQL

JSON — текст, JSONB — бинарный (индексируемый, быстрее).

```sql
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    data JSONB
);

-- Вставка
INSERT INTO users (data) VALUES ('{"name": "Sergey", "age": 35, "skills": ["SQL", "Python"]}');

-- Запросы
SELECT data->>'name' AS name,
       data->'age' AS age,
       data->'skills'->0 AS first_skill
FROM users;

-- Поиск
SELECT * FROM users WHERE data @> '{"age": 35}';
SELECT * FROM users WHERE data->'skills' ? 'SQL';
```

**Индексы на JSONB**
```sql
CREATE INDEX idx_users_name ON users ((data->>'name'));
CREATE INDEX idx_users_skills ON users USING GIN ((data->'skills'));
CREATE INDEX idx_users_gin ON users USING GIN (data);
```

### 11.2. Массивы

```sql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    tags TEXT[]
);

INSERT INTO products (tags) VALUES ('{electronics, laptop, apple}');

-- Запросы
SELECT * FROM products WHERE 'laptop' = ANY(tags);
SELECT * FROM products WHERE tags @> '{apple}';  -- содержит
SELECT * FROM products WHERE tags && '{apple, samsung}';  -- пересечение
```

**Индекс на массив**
```sql
CREATE INDEX idx_products_tags ON products USING GIN (tags);
```

### 11.3. Materialized Views

Физическая копия запроса — ускоряет чтение, но требует обновления.

```sql
CREATE MATERIALIZED VIEW daily_sales AS
SELECT DATE_TRUNC('day', sale_date) AS day,
       SUM(amount) AS total
FROM sales
GROUP BY 1;

-- Обновление
REFRESH MATERIALIZED VIEW daily_sales;
-- Обновление без блокировки чтения
REFRESH MATERIALIZED VIEW CONCURRENTLY daily_sales;
```

### 11.4. LATERAL JOIN

Для подзапросов, зависящих от предыдущей таблицы.

```sql
SELECT u.id, u.name, top_order.id AS top_order_id
FROM users u
CROSS JOIN LATERAL (
    SELECT id, amount
    FROM orders
    WHERE user_id = u.id
    ORDER BY amount DESC
    LIMIT 1
) top_order;
```

### 11.5. Генерация рядов (generate_series)

```sql
-- Числа от 1 до 10
SELECT generate_series(1, 10) AS n;

-- Даты за месяц
SELECT generate_series('2024-01-01'::date, '2024-01-31'::date, '1 day'::interval) AS dt;
```

### 11.6. Pivot / Crosstab (разворот таблицы)

```sql
-- PostgreSQL: crosstab
CREATE EXTENSION tablefunc;

SELECT * FROM crosstab(
    'SELECT department, EXTRACT(YEAR FROM hire_date)::text, COUNT(*)
     FROM employees GROUP BY 1, 2 ORDER BY 1, 2',
    'SELECT DISTINCT EXTRACT(YEAR FROM hire_date)::text FROM employees ORDER BY 1'
) AS ct(department text, "2022" int, "2023" int, "2024" int);
```

**Шпаргалка раздела 11**
```sql
-- JSONB
data->>'key'
data @> '{"key": "value"}'
data->'array' ? 'value'

-- Массивы
col @> '{value}'
'value' = ANY(col)

-- Generate series
generate_series(1, 10)
generate_series(date1, date2, '1 day')

-- LATERAL
CROSS JOIN LATERAL (SELECT ...) t
```

## РАЗДЕЛ 12: ПРАКТИЧЕСКИЕ ПАТТЕРНЫ И ЗАДАЧИ

### 12.1. Поиск дубликатов
```sql
SELECT email, phone, COUNT(*) AS dup_count
FROM users
GROUP BY email, phone
HAVING COUNT(*) > 1;
```

### 12.2. Иерархические данные (дерево сотрудников)
```sql
WITH RECURSIVE hierarchy AS (
    SELECT id, name, manager_id, 1 AS level
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.id, e.name, e.manager_id, h.level + 1
    FROM employees e
    JOIN hierarchy h ON e.manager_id = h.id
)
SELECT * FROM hierarchy;
```

### 12.3. Gaps and Islands (группы последовательных значений)
```sql
WITH ordered AS (
    SELECT 
        user_id,
        login_date,
        login_date - ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY login_date) AS grp
    FROM logins
)
SELECT 
    user_id,
    MIN(login_date) AS streak_start,
    MAX(login_date) AS streak_end,
    COUNT(*) AS streak_length
FROM ordered
GROUP BY user_id, grp
ORDER BY user_id, streak_start;
```

### 12.4. Скользящие средние и накопительные суммы
```sql
-- Накопительная сумма
SELECT sale_date, amount,
       SUM(amount) OVER (ORDER BY sale_date) AS running_total
FROM sales;

-- Скользящее среднее 7 дней
SELECT sale_date, revenue,
       AVG(revenue) OVER (
           ORDER BY sale_date
           ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
       ) AS ma_7d
FROM daily_revenue;
```

### 12.5. Топ-N в группе
```sql
WITH ranked AS (
    SELECT 
        category,
        product,
        sales,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales DESC) AS rn
    FROM sales_data
)
SELECT * FROM ranked WHERE rn <= 3;
```

### 12.6. Поиск отсутствующих данных (пропущенные даты)
```sql
WITH all_dates AS (
    SELECT generate_series('2024-01-01'::date, '2024-01-31'::date, '1 day') AS dt
)
SELECT dt, COALESCE(sales, 0) AS sales
FROM all_dates d
LEFT JOIN sales s ON d.dt = s.sale_date
ORDER BY dt;
```

**Антипаттерны раздела 12**

❌ Само-соединение для топ-N вместо оконных функций
```sql
-- ПЛОХО: медленно и сложно
SELECT t1.* FROM sales t1
WHERE 3 > (SELECT COUNT(*) FROM sales t2
           WHERE t2.category = t1.category AND t2.sales > t1.sales);
```

✅ ХОРОШО
```sql
ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales DESC) rn
```

❌ Цикл в рекурсии без защиты
```sql
WITH RECURSIVE cte AS (...)  -- без level < 100
```

❌ DISTINCT вместо оконных функций для last_value
```sql
-- ПЛОХО
SELECT DISTINCT ON (user_id) * ORDER BY user_id, time DESC
```
(хорошо, но оконные функции часто гибче)

**Лучшие практики раздела 12**

✅ Всегда используй оконные функции для ранжирования/кумулятивных расчётов
✅ Для топ-N — ROW_NUMBER() + CTE, а не само-соединение
✅ Для последовательных групп — date - ROW_NUMBER()
✅ Для заполнения пропусков — generate_series + LEFT JOIN
✅ Для иерархий — рекурсивный CTE с ограничением уровня
✅ Тестируй на больших данных — многие паттерны требуют индексов

**Различия MySQL и PostgreSQL**

| Функция/паттерн          | MySQL                          | PostgreSQL                     |
|---------------------------|--------------------------------|--------------------------------|
| Топ-N                     | Переменные или подзапросы      | DISTINCT ON / оконные функции  |
| Рекурсия                  | Нет нативно (до 8.0)           | RECURSIVE CTE                  |
| Generate_series           | Нет                            | generate_series()              |
| Crosstab/pivot            | CASE + GROUP BY                | tablefunc.crosstab()           |

**Шпаргалка раздела 12**
```sql
-- Топ-N
ROW_NUMBER() OVER (PARTITION BY group ORDER BY val DESC) rn

-- Running total
SUM(val) OVER (ORDER BY date ROWS UNBOUNDED PRECEDING)

-- Gap & Island
date - ROW_NUMBER() OVER (ORDER BY date) AS grp

-- Dedup
DELETE WHERE id NOT IN (SELECT MAX(id) FROM ... GROUP BY dup_key)

-- Заполнить пропуски
generate_series(date1, date2, '1 day') dt LEFT JOIN ...

-- Последняя запись
DISTINCT ON (user_id) * ORDER BY user_id, time DESC
```

## ПРИЛОЖЕНИЕ A: Различия MySQL и PostgreSQL

### A.1. Основные различия

| Характеристика | MySQL | PostgreSQL |
|----------------|-------|------------|
| Тип СУБД | Реляционная | Реляционно-объектная |
| Лицензия | GPL (Oracle) | PostgreSQL License (BSD-подобная) |
| Стандарт SQL | Не всегда строгий | Более строгое соответствие |
| Производительность | Оптимизирован для чтения | Оптимизирован для сложных запросов |
| Репликация | Встроенная (master-slave) | Физическая и логическая |

### A.2. Типы данных

| Тип данных | MySQL | PostgreSQL |
|------------|-------|------------|
| Логический тип | BOOLEAN (синоним TINYINT(1)) | BOOLEAN (нативный) |
| Массивы | Не поддерживаются | data_type[] (нативно) |
| JSON | JSON (с 5.7) | JSON и JSONB (бинарный) |
| Псевдонимы типов | SERIAL = INT AUTO_INCREMENT | SERIAL = INT + SEQUENCE |
| Enum | ENUM('val1', 'val2') | ENUM (через CREATE TYPE) |

### A.3. Синтаксические различия
**Ограничения длины строки:**
```sql
-- MySQL
CREATE TABLE t (name VARCHAR(255)); -- Обязательно указывать длину
-- PostgreSQL
CREATE TABLE t (name TEXT); -- TEXT без ограничения длины
```

**Автоинкремент:**
```sql
-- MySQL
CREATE TABLE t (id INT PRIMARY KEY AUTO_INCREMENT);
-- PostgreSQL
CREATE TABLE t (id SERIAL PRIMARY KEY);
```

**LIMIT и OFFSET:**
```sql
-- MySQL
SELECT * FROM t LIMIT 10 OFFSET 20;
SELECT * FROM t LIMIT 20, 10;
-- PostgreSQL
SELECT * FROM t LIMIT 10 OFFSET 20;
```

### A.4. Работа с датами
**Текущая дата и время:**
```sql
-- MySQL
SELECT NOW();
-- PostgreSQL
SELECT NOW();
```

**Форматирование дат:**
```sql
-- MySQL
SELECT DATE_FORMAT(NOW(), '%Y-%m-%d %H:%i:%s');
-- PostgreSQL
SELECT TO_CHAR(NOW(), 'YYYY-MM-DD HH24:MI:SS');
```

### A.5. Строковые функции
**Конкатенация:**
```sql
-- MySQL
SELECT CONCAT('Hello', ' ', 'World');
-- PostgreSQL
SELECT 'Hello' || ' ' || 'World';
```

### A.6. Оконные функции
- MySQL: полная поддержка с версии 8.0
- PostgreSQL: полная поддержка давно

### A.7. Индексы
```sql
-- PostgreSQL
CREATE INDEX idx_covering ON table(col1, col2) INCLUDE (col3);
```

### A.8. Хранимые процедуры и функции
```sql
-- MySQL
CREATE PROCEDURE get_user(IN user_id INT)
BEGIN
    SELECT * FROM users WHERE id = user_id;
END;
```

### A.9. Работа с транзакциями
- MySQL (InnoDB): по умолчанию REPEATABLE READ
- PostgreSQL: по умолчанию READ COMMITTED

### A.10. Системные функции
```sql
-- MySQL
SHOW TABLES;
-- PostgreSQL
\dt
```

**Шпаргалка приложения A**
```sql
-- PostgreSQL
SERIAL, TIMESTAMPTZ, JSONB, generate_series, TO_CHAR
-- MySQL
AUTO_INCREMENT, DATETIME, JSON, DATE_FORMAT
```

## ПРИЛОЖЕНИЕ B: Шпаргалка по функциям

### B.1. Агрегатные функции
| Функция | Описание | Пример |
|---------|----------|--------|
| COUNT() | Подсчет строк | COUNT(*), COUNT(column) |
| SUM() | Сумма значений | SUM(price) |
| AVG() | Среднее значение | AVG(salary) |
| MIN() | Минимальное значение | MIN(age) |
| MAX() | Максимальное значение | MAX(temperature) |
| GROUP_CONCAT() | Объединение строк (MySQL) | GROUP_CONCAT(name SEPARATOR ', ') |
| STRING_AGG() | Объединение строк (PostgreSQL) | STRING_AGG(name, ', ') |

### B.2. Строковые функции
| Функция | MySQL | PostgreSQL |
|---------|-------|------------|
| Конкатенация | CONCAT(s1, s2) | s1 \|\| s2 или CONCAT(s1, s2) |
| Длина строки | CHAR_LENGTH(str) | LENGTH(str) |
| Верхний регистр | UPPER(str) | UPPER(str) |
| Обрезка пробелов | TRIM(str) | TRIM(str) |
| Подстрока | SUBSTRING(str, pos, len) | SUBSTRING(str FROM pos FOR len) |
| Замена | REPLACE(str, old, new) | REPLACE(str, old, new) |

### B.3. Функции даты и времени
| Функция | MySQL | PostgreSQL |
|---------|-------|------------|
| Текущая дата/время | NOW(), CURDATE() | NOW(), CURRENT_DATE |
| Добавление интервала | DATE_ADD(date, INTERVAL 1 DAY) | date + INTERVAL '1 day' |
| Разница дат | DATEDIFF(date1, date2) | date1 - date2 |
| Извлечение части | YEAR(date), MONTH(date) | EXTRACT(YEAR FROM date) |
| Форматирование | DATE_FORMAT(date, '%Y-%m-%d') | TO_CHAR(date, 'YYYY-MM-DD') |

### B.4. Числовые функции
| Функция | Описание | Пример |
|---------|----------|--------|
| ROUND() | Округление | ROUND(price, 2) |
| CEIL() | Округление вверх | CEIL(4.1) → 5 |
| FLOOR() | Округление вниз | FLOOR(4.9) → 4 |
| ABS() | Абсолютное значение | ABS(-5) → 5 |

### B.5. Функции для работы с NULL
| Функция | Описание | Пример |
|---------|----------|--------|
| COALESCE() | Первое не-NULL значение | COALESCE(phone, email, 'N/A') |
| IFNULL() (MySQL) | Замена NULL | IFNULL(column, 'default') |
| NULLIF() | NULL если равны | NULLIF(column, 0) |

### B.6. Оконные функции
| Функция | Категория | Описание |
|---------|-----------|----------|
| ROW_NUMBER() | Ранжирование | Уникальный номер строки |
| RANK() | Ранжирование | Ранг с пропусками |
| DENSE_RANK() | Ранжирование | Ранг без пропусков |
| LAG()/LEAD() | Смещение | Предыдущая/следующая строка |
| SUM() OVER() | Агрегация | Сумма по окну |

### B.7. JSON функции (PostgreSQL JSONB)
| Функция | Описание | Пример |
|---------|----------|--------|
| -> | Получить JSON объект | json_column->'key' |
| ->> | Получить текст | json_column->>'key' |
| @> | Содержит | json_column @> '{"key": "value"}'::jsonb |
| jsonb_set() | Установить значение | jsonb_set(json_column, '{key}', '"value"') |

## ПРИЛОЖЕНИЕ C: Чек-лист оптимизации запросов

### C.1. Перед написанием запроса
- [ ] Понимание данных: структура таблиц, типы данных, отношения
- [ ] Цель запроса: четко понимаю, что нужно получить
- [ ] Объем данных: представляю количество строк

### C.2. Написание запроса
**Структура и читаемость:**
- [ ] Форматирование (отступы, переносы)
- [ ] Понятные алиасы
- [ ] Комментарии к сложным частям
- [ ] Разбиение на CTE

**Производительность:**
- [ ] Только нужные столбцы (не SELECT *)
- [ ] Фильтрация как можно раньше
- [ ] EXISTS вместо IN для подзапросов
- [ ] Без функций в WHERE на индексных полях
- [ ] UNION ALL вместо UNION

### C.3. Анализ запроса
- [ ] EXPLAIN / EXPLAIN ANALYZE
- [ ] Тип доступа (index scan лучше seq scan)
- [ ] Количество строк
- [ ] Временные таблицы / filesort

### C.4. Оптимизация
**Индексы:**
- [ ] Индексы на WHERE, JOIN, ORDER BY
- [ ] Составные индексы
- [ ] Покрывающие индексы
- [ ] Удаление неиспользуемых индексов

**Переписывание:**
- [ ] Подзапрос → JOIN
- [ ] Материализация CTE
- [ ] Партиционирование больших таблиц

### C.5. Тестирование
- [ ] На реальных объемах
- [ ] Конкурентная нагрузка

### C.6. Мониторинг
- [ ] Логи медленных запросов
- [ ] Статистика индексов
- [ ] Рост таблиц

### C.7. Частые проблемы
| Проблема | Решение |
|----------|---------|
| Медленный запрос | EXPLAIN + индексы + переписать |
| Высокая CPU | Оптимизировать запросы |
| Много I/O | Покрывающие индексы |
| Deadlocks | Короткие транзакции, одинаковый порядок |

### C.8. Команды диагностики
**MySQL**
```sql
SHOW FULL PROCESSLIST;
SELECT * FROM sys.schema_unused_indexes;
```
**PostgreSQL**
```sql
SELECT * FROM pg_stat_activity WHERE state = 'active';
SELECT schemaname, tablename, indexname, idx_scan FROM pg_stat_user_indexes WHERE idx_scan = 0;
```

### C.9. Рекомендуемые настройки для производительности

**MySQL**
```ini
# my.cnf
innodb_buffer_pool_size = 70% of RAM
innodb_log_file_size = 1-2GB
innodb_flush_log_at_trx_commit = 2  # для лучшей производительности
innodb_file_per_table = ON
```

**PostgreSQL**
```ini
# postgresql.conf
shared_buffers = 25% of RAM
effective_cache_size = 50-75% of RAM
work_mem = 64MB  # для сортировок и хэшей
maintenance_work_mem = 1GB  # для обслуживания
```

# Конец плейбука

Если нужно добавить/изменить — пиши PR или просто скажи.